# Import libraries

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Linear Regression Model").getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Load and verify data

In [3]:
data = spark.read.csv('resources/Ecommerce_Customers.csv',header = True, inferSchema = True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.head(3)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264),
 Row(Email='pallen@yahoo.com', Address='24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564', Avatar='Bisque', Avg Session Length=33.000914755642675, Time on App=11.330278057777512, Time on Website=37.110597442120856, Length of Membership=4.104543202376424, Yearly Amount Spent=487.54750486747207)]

In [6]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

# Train Test split

In [8]:
assembler = VectorAssembler(inputCols =['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
print(output.features)

Column<b'features'>


In [12]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [13]:
final_data = output.select('features','Yearly Amount Spent')

In [14]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                340|
|   mean|  500.9202135907162|
| stddev|  75.65389681262481|
|    min| 256.67058229005585|
|    max|  712.3963268096637|
+-------+-------------------+



In [16]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                160|
|   mean|  495.9009156778252|
| stddev|    86.733699718267|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



# Build Model

In [17]:
regressor = LinearRegression(labelCol='Yearly Amount Spent')

In [18]:
model = regressor.fit(train_data)

# Evaluate Model

In [19]:
pred_data = model.evaluate(test_data)

In [20]:
pred_data.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -13.62753285676149|
| -1.318687682785992|
|-19.049874272699526|
| -3.796175410594799|
|  5.768771611463421|
|-22.035629927494938|
| 1.8187791960635877|
|-3.5910837249108454|
| -5.390329230392467|
| -6.740035447163734|
|-2.3510306283217233|
|-6.0447514688781325|
|  0.954739428555456|
|-11.779892871444304|
| -5.568074390509423|
| -16.78692264001046|
| -9.797722129248143|
|-0.6034664472291524|
|-3.0034861289760784|
| -9.515544345484727|
+-------------------+
only showing top 20 rows



In [21]:
pred_data.rootMeanSquaredError

9.80909953607503

In [22]:
pred_data.r2

0.9871292042882338

In [23]:
unlabeled_data = test_data.select('features')
test_predictions = model.transform(unlabeled_data)

In [24]:
test_predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 333.5564026599551|
|[30.5743636841713...|443.38310144085165|
|[30.8162006488763...| 285.1362152211685|
|[30.8794843441274...| 494.0027753954495|
|[30.9716756438877...| 488.8698381454293|
|[31.1239743499119...| 508.9826837672607|
|[31.3091926408918...|430.90193864387004|
|[31.4252268808548...| 534.3578023796728|
|[31.5147378578019...| 495.2028172268539|
|[31.5171218025062...|282.65845609754945|
|[31.6610498227460...|418.70938420822256|
|[31.7242025238451...| 509.4326387568386|
|[31.7366356860502...|495.97870682697646|
|[31.8093003166791...| 548.5517922342855|
|[31.8124825597242...|398.37841937430665|
|[31.8164283341993...| 517.9094141436668|
|[31.8279790554652...|449.80046967618966|
|[31.8293464559211...|385.75580443520414|
|[31.8530748017465...|462.28860959132805|
|[32.0085045178551...| 452.7127653742401|
+--------------------+------------